In [1]:
import pandas as pd
import networkx as nx
import numpy as np

In [2]:
simplify_dir = 'GTFS/stop_snapping/GIS Simplify/'
gtfs_dir = 'GTFS/TTC_2016-10-03/'

In [3]:
stop_times_reduced = pd.read_csv(gtfs_dir + 'stop_times_reduced.csv')
stop_times_reduced

,trip_id,stop_sequence_new,stop_sequence,hr,min,INT_ID
0,32828689,1,1,5,15,13466640.0
1,32828689,2,2,5,16,13466531.0
2,32828689,3,3,5,16,13466677.0
3,32828689,4,4,5,17,13466754.0
4,32828689,5,5,5,18,13466699.0
...,...,...,...,...,...,...
3959705,32970576,7,8,25,56,13461602.0
3959706,32970576,8,9,25,57,13461912.0
3959707,32970576,9,10,25,57,13462284.0
3959708,32970576,10,11,25,58,13462427.0


In [4]:
hr_list = [17,18,19,20,21]
departure_hour = [17]

In [5]:
stop_times_reduced = stop_times_reduced[stop_times_reduced['hr'].isin(hr_list)]

stop_times_reduced['hr'] = np.where(stop_times_reduced['hr']>23, stop_times_reduced['hr'] - 24, stop_times_reduced['hr'])


/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_6467/1662348926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stop_times_reduced['hr'] = np.where(stop_times_reduced['hr']>23, stop_times_reduced['hr'] - 24, stop_times_reduced['hr'])


In [6]:
stop_times_reduced_o = stop_times_reduced.copy()
stop_times_reduced_d = stop_times_reduced.copy()
stop_times_reduced_d['stop_sequence_new'] = stop_times_reduced_d['stop_sequence_new'] - 1

In [7]:
stop_times_od = stop_times_reduced_o.merge(stop_times_reduced_d, left_on = ['trip_id', 'stop_sequence_new'],
                           right_on = ['trip_id', 'stop_sequence_new'], suffixes = ['_o', '_d'])

In [8]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d
0,32828765,53,57,17,1,13461057.0,58,17,2,13460922.0
1,32828765,54,58,17,2,13460922.0,59,17,3,13460822.0
2,32828765,55,59,17,3,13460822.0,60,17,6,225.0
3,32828766,49,53,17,1,13461912.0,54,17,2,13461602.0
4,32828766,50,54,17,2,13461602.0,55,17,3,13461421.0
...,...,...,...,...,...,...,...,...,...,...
966857,32970419,11,13,19,13,13463440.0,14,19,14,13463074.0
966858,32970419,12,14,19,14,13463074.0,15,19,15,13462698.0
966859,32970419,13,15,19,15,13462698.0,16,19,16,13462239.0
966860,32970419,14,16,19,16,13462239.0,17,19,17,13461829.0


In [9]:
stop_times_od['min_o'] = stop_times_od['min_o'].apply(lambda x: '{0:0>2}'.format(x))
stop_times_od['min_d'] = stop_times_od['min_d'].apply(lambda x: '{0:0>2}'.format(x))

In [10]:
stop_times_od['hr_o'] = stop_times_od['hr_o'].astype(str)
stop_times_od['hr_d'] = stop_times_od['hr_d'].astype(str)
# stop_times_od['min_o'] = stop_times_od['min_o'].astype(str)

In [11]:
stop_times_od['o_time_str'] = stop_times_od['hr_o'] + ':' + stop_times_od['min_o']
stop_times_od['d_time_str'] = stop_times_od['hr_d'] + ':' + stop_times_od['min_d']

In [12]:
stop_times_od['o_time'] = pd.to_datetime(stop_times_od['o_time_str'], format = '%H:%M')
stop_times_od['d_time'] = pd.to_datetime(stop_times_od['d_time_str'], format = '%H:%M')


In [13]:
stop_times_od['cost'] = ((stop_times_od['d_time'] - stop_times_od['o_time']).dt.seconds)/60

In [14]:
stop_times_od['o_time'] = stop_times_od['o_time'].dt.time
stop_times_od['d_time'] = stop_times_od['d_time'].dt.time

In [15]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,o_time_str,d_time_str,o_time,d_time,cost
0,32828765,53,57,17,01,13461057.0,58,17,02,13460922.0,17:01,17:02,17:01:00,17:02:00,1.0
1,32828765,54,58,17,02,13460922.0,59,17,03,13460822.0,17:02,17:03,17:02:00,17:03:00,1.0
2,32828765,55,59,17,03,13460822.0,60,17,06,225.0,17:03,17:06,17:03:00,17:06:00,3.0
3,32828766,49,53,17,01,13461912.0,54,17,02,13461602.0,17:01,17:02,17:01:00,17:02:00,1.0
4,32828766,50,54,17,02,13461602.0,55,17,03,13461421.0,17:02,17:03,17:02:00,17:03:00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966857,32970419,11,13,19,13,13463440.0,14,19,14,13463074.0,19:13,19:14,19:13:00,19:14:00,1.0
966858,32970419,12,14,19,14,13463074.0,15,19,15,13462698.0,19:14,19:15,19:14:00,19:15:00,1.0
966859,32970419,13,15,19,15,13462698.0,16,19,16,13462239.0,19:15,19:16,19:15:00,19:16:00,1.0
966860,32970419,14,16,19,16,13462239.0,17,19,17,13461829.0,19:16,19:17,19:16:00,19:17:00,1.0


In [16]:
trips_weekday = pd.read_csv(gtfs_dir + 'trips_weekday.csv')

In [17]:
stop_times_od = stop_times_od.merge(trips_weekday[['trip_id', 'route_short_name', 'direction_id']], how = 'inner')

In [18]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,o_time_str,d_time_str,o_time,d_time,cost,route_short_name,direction_id
0,32828765,53,57,17,01,13461057.0,58,17,02,13460922.0,17:01,17:02,17:01:00,17:02:00,1.0,506,0
1,32828765,54,58,17,02,13460922.0,59,17,03,13460822.0,17:02,17:03,17:02:00,17:03:00,1.0,506,0
2,32828765,55,59,17,03,13460822.0,60,17,06,225.0,17:03,17:06,17:03:00,17:06:00,3.0,506,0
3,32828766,49,53,17,01,13461912.0,54,17,02,13461602.0,17:01,17:02,17:01:00,17:02:00,1.0,506,0
4,32828766,50,54,17,02,13461602.0,55,17,03,13461421.0,17:02,17:03,17:02:00,17:03:00,1.0,506,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301765,32970419,11,13,19,13,13463440.0,14,19,14,13463074.0,19:13,19:14,19:13:00,19:14:00,1.0,35,1
301766,32970419,12,14,19,14,13463074.0,15,19,15,13462698.0,19:14,19:15,19:14:00,19:15:00,1.0,35,1
301767,32970419,13,15,19,15,13462698.0,16,19,16,13462239.0,19:15,19:16,19:15:00,19:16:00,1.0,35,1
301768,32970419,14,16,19,16,13462239.0,17,19,17,13461829.0,19:16,19:17,19:16:00,19:17:00,1.0,35,1


Inner merge because some trips in the stop times files have `service_id` 2 and 3 (meaning they run on weekends).

Type 1 nodes are boarding, while type 2 nodes are alighting. Type 0 nodes are regular waiting nodes. This is to penalize transfers, and make transfering more realistic. Links connecting type 1 and type 2 nodes to type 0 nodes will be added. Type 3 nodes are destination constrained nodes (to distinguish from origin zones)

In [19]:
stop_times_od['node_o'] = stop_times_od['INT_ID_o'].astype(
    int).astype(str) + '-' + stop_times_od['route_short_name'].astype(int).astype(str) +'-'+ stop_times_od['hr_o'].astype(
    str) + '-' + stop_times_od['min_o'].astype(int).astype(str)
stop_times_od['node_d'] = stop_times_od['INT_ID_d'].astype(
    int).astype(str) + '-' + stop_times_od['route_short_name'].astype(int).astype(str) +'-'+ stop_times_od['hr_d'].astype(
    str) + '-' + stop_times_od['min_d'].astype(int).astype(str)

In [20]:
stop_times_od['link'] = '0' + '-' + stop_times_od['route_short_name'].astype(int).astype(str) +'-' + stop_times_od['trip_id'].astype(str) + '-' + stop_times_od['direction_id'].astype(str)+ '_' + stop_times_od['node_o'] + '_' + stop_times_od['node_d']

In [21]:
stop_times_od['cost'] = stop_times_od['cost'].astype(int)

In [22]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,o_time_str,d_time_str,o_time,d_time,cost,route_short_name,direction_id,node_o,node_d,link
0,32828765,53,57,17,01,13461057.0,58,17,02,13460922.0,17:01,17:02,17:01:00,17:02:00,1,506,0,13461057-506-17-1,13460922-506-17-2,0-506-32828765-0_13461057-506-17-1_13460922-50...
1,32828765,54,58,17,02,13460922.0,59,17,03,13460822.0,17:02,17:03,17:02:00,17:03:00,1,506,0,13460922-506-17-2,13460822-506-17-3,0-506-32828765-0_13460922-506-17-2_13460822-50...
2,32828765,55,59,17,03,13460822.0,60,17,06,225.0,17:03,17:06,17:03:00,17:06:00,3,506,0,13460822-506-17-3,225-506-17-6,0-506-32828765-0_13460822-506-17-3_225-506-17-6
3,32828766,49,53,17,01,13461912.0,54,17,02,13461602.0,17:01,17:02,17:01:00,17:02:00,1,506,0,13461912-506-17-1,13461602-506-17-2,0-506-32828766-0_13461912-506-17-1_13461602-50...
4,32828766,50,54,17,02,13461602.0,55,17,03,13461421.0,17:02,17:03,17:02:00,17:03:00,1,506,0,13461602-506-17-2,13461421-506-17-3,0-506-32828766-0_13461602-506-17-2_13461421-50...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301765,32970419,11,13,19,13,13463440.0,14,19,14,13463074.0,19:13,19:14,19:13:00,19:14:00,1,35,1,13463440-35-19-13,13463074-35-19-14,0-35-32970419-1_13463440-35-19-13_13463074-35-...
301766,32970419,12,14,19,14,13463074.0,15,19,15,13462698.0,19:14,19:15,19:14:00,19:15:00,1,35,1,13463074-35-19-14,13462698-35-19-15,0-35-32970419-1_13463074-35-19-14_13462698-35-...
301767,32970419,13,15,19,15,13462698.0,16,19,16,13462239.0,19:15,19:16,19:15:00,19:16:00,1,35,1,13462698-35-19-15,13462239-35-19-16,0-35-32970419-1_13462698-35-19-15_13462239-35-...
301768,32970419,14,16,19,16,13462239.0,17,19,17,13461829.0,19:16,19:17,19:16:00,19:17:00,1,35,1,13462239-35-19-16,13461829-35-19-17,0-35-32970419-1_13462239-35-19-16_13461829-35-...


In [23]:
walk_links = pd.read_csv('GIS/int_tts_walk_time.csv')
walk_links['duration'] = round(walk_links['duration']/60).astype(int)
walk_links['gta06'] = walk_links['gta06'] + 1000
walk_links

,gta06,INT_ID,duration
0,1001,13461602,25
1,1001,13461733,21
2,1001,13462844,20
3,1001,13461863,27
4,1001,13463408,9
...,...,...,...
37157,1081,13467412,24
37158,1081,13467854,6
37159,1081,13466312,22
37160,1081,13467197,29


In [24]:
G = nx.DiGraph()

## Adding node-times

In [25]:
nodes = list((stop_times_od['INT_ID_o'].append(stop_times_od['INT_ID_d'])).drop_duplicates())

In [26]:
node_list = []
for node in nodes:
    for hr_i in hr_list:
        if hr_i > 23:  
            hr = hr_i - 24
        else:
            hr = hr_i
        for minute in range(60):
            node_id = str(int(node)) + '-0-' + str(hr) + '-' + str(minute)
            node_list.append(node_id)

In [27]:
G.add_nodes_from(node_list)

## Adding Boarding Nodes

In [28]:
boarding_nodes = list(stop_times_od['node_o'].drop_duplicates())

In [29]:
G.add_nodes_from(boarding_nodes)

## Adding Alighting Nodes

In [30]:
alighting_nodes = list(stop_times_od['node_d'].drop_duplicates())

In [31]:
G.add_nodes_from(alighting_nodes)

## Adding travel links

In [32]:
link_list = []
for index, row in stop_times_od.iterrows():
    attributes = {'cost': row['cost']}
    link = (row['node_o'], row['node_d'], attributes)
    link_list.append(link)

In [33]:
G.add_edges_from(link_list)

## Adding waiting links

In [34]:
transfer_list = []
for node in nodes:
    next_hr = hr_list[0]
    next_minute = 0
    node = int(node)
    for hr_i in hr_list:
        if hr_i > 23:
            current_hr = hr_i - 24 
        else:
            current_hr = hr_i
        for minute in range(60):
            current_minute = minute
            if current_minute == 59:
                next_minute = 0
                next_hr = current_hr + 1
                
            else:
                next_minute = current_minute + 1
            node_o = str(node) + '-0-' + str(current_hr) + '-' + str(current_minute)
            node_d = str(node) + '-0-' + str(next_hr) + '-' + str(next_minute)
            
            link_name = '1-0-0-9_' + node_o + '_' + node_d
            
            attributes = {'cost': 1}
            
            
            link = (node_o, node_d, attributes)
            transfer_list.append(link)
    

In [35]:
G.add_edges_from(transfer_list)

## Adding Boarding Links

As a start, we'll match Google Maps minimum transfer time of 4 minutes, with 2 minute cost for both boarding and alighting links, meaning transfers be a minimum of 4 minutes.

A direction id of 1 means inbound to the stop node.

In [36]:
boarding_link = []
for node in boarding_nodes:
    node_split = node.split('-')
    int_id = node_split[0]
    boarding_hr = int(node_split[2])
    boarding_minute = int(node_split[3])

    stop_minute = boarding_minute - 2
    if stop_minute < 0:
        stop_hr = boarding_hr -1
        stop_minute = 60 + stop_minute
    else:
        stop_hr = boarding_hr

    stop_node = int_id + '-0-' + str(stop_hr) + '-' + str(stop_minute)
    attributes = {'cost': 2}
    
    link = (stop_node, node, attributes)
    
    boarding_link.append(link)

In [37]:
G.add_edges_from(boarding_link)

## Adding Alighting Links

In [38]:
alighting_link = []
for node in alighting_nodes:
    node_split = node.split('-')
    int_id = node_split[0]
    alighting_hr = int(node_split[2])
    alighting_minute = int(node_split[3])

    stop_minute = alighting_minute + 2
    if stop_minute > 59:
        stop_hr = alighting_hr + 1
        stop_minute = stop_minute - 60
    else:
        stop_hr = alighting_hr

    stop_node = int_id + '-0-' + str(stop_hr) + '-' + str(stop_minute)
    attributes = {'cost': 2}
    
    link = (node, stop_node, attributes)
    
    alighting_link.append(link)

In [39]:
G.add_edges_from(alighting_link)

## Adding TTS Zone Origins

Will also follow node-time structure

In [40]:
tts_zones = list(walk_links['gta06'].drop_duplicates())

In [41]:
node_stationary_list = []
for zone in tts_zones:
    for hr_i in hr_list:
        if hr_i > 23:  
            hr = hr_i - 24
        else:
            hr = hr_i
        for minute in range(60):
            node_id = str(int(zone)) + '-0-' + str(hr) + '-' + str(minute)
            node_stationary_list.append(node_id)

In [42]:
G.add_nodes_from(node_stationary_list)

## Adding TTS Zone Destinations

Marked by type 3 nodes, will also follow node-time structure

In [43]:
node_dest_list = []
for zone in tts_zones:
    for hr_i in hr_list:
        if hr_i > 23:  
            hr = hr_i - 24
        else:
            hr = hr_i
        for minute in range(60):
            node_id = str(int(zone)) + '-3-' + str(hr) + '-' + str(minute)
            node_dest_list.append(node_id)

In [44]:
G.add_nodes_from(node_dest_list)

## Adding links from TTS to intersections

Both outbound and inbound links will be added. In pathfinding process, only inbound links for the destination, and outbound links for the origin will be kept.

Only links during the departure hour will be made due to space concerns.

Convention is 0 for outbound and 1 for inbound

In [45]:
walk_links.head()

,gta06,INT_ID,duration
0,1001,13461602,25
1,1001,13461733,21
2,1001,13462844,20
3,1001,13461863,27
4,1001,13463408,9


In [46]:
walk_link_outbound = []
for index, row in walk_links.iterrows():
    
    zone = int(row['gta06'])
    node = int(row['INT_ID'])
    cost = int(row['duration'])
    
    for hr_i in departure_hour:
        if hr_i > 23:
            current_hr = hr_i - 24 
        else:
            current_hr = hr_i
        for minute in range(60):
            current_minute = minute
            next_hr = current_hr
            next_minute = current_minute + cost
            if next_minute > 59:
                next_hr= current_hr + 1
                next_minute = next_minute - 60
                
            node_o = str(zone) + '-0-' + str(current_hr) + '-' + str(current_minute)
            node_d = str(node) + '-0-' + str(next_hr) + '-' + str(next_minute)
            
            link_name = '2-0-0-0_' + node_o + '_' + node_d
            
            attributes = {'cost': cost}
    
            link = (node_o, node_d, attributes)
            walk_link_outbound.append(link)

## Adding links from intersections to TTS Zones

In [47]:
walk_link_inbound = []
for index, row in walk_links.iterrows():
    
    zone = int(row['gta06'])
    node = int(row['INT_ID'])
    cost = int(row['duration'])
    
    for hr_i in hr_list:
        if hr_i > 23:
            current_hr = hr_i - 24 
        else:
            current_hr = hr_i
        for minute in range(60):
            current_minute = minute
            next_hr = current_hr
            next_minute = current_minute + cost
            if next_minute > 59:
                next_hr= current_hr + 1
                next_minute = next_minute - 60
                
            node_o = str(node) + '-0-' + str(current_hr) + '-' + str(current_minute)
            node_d = str(zone) + '-9999-' + str(next_hr) + '-' + str(next_minute)
            
            link_name = '2-0-0-1_' + node_o + '_' + node_d
            
            attributes = {'cost': cost}
    
            link = (node_o, node_d, attributes)
            walk_link_inbound.append(link)

In [48]:
G.add_edges_from(walk_link_outbound)

In [49]:
G.add_edges_from(walk_link_inbound)

In [50]:
nx.write_graphml(G,'networks/PM-TE-16-5-constrained.graphml')
stop_times_od.to_csv('networks/PM-TE-16-5-constrained.csv', index = False)

In [51]:
G.number_of_nodes()

1934657

In [52]:
G.number_of_edges()

15295222